In [13]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [14]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [15]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [16]:
result = data_train.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [19]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = random.choices(items, weights=weights, k=n)
    
    return recs

In [20]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights = items_weights[['item_id', 'weight']]

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 39.1 s, sys: 866 ms, total: 39.9 s
Wall time: 40.2 s


,user_id,actual,weighted_random_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[1136958, 9859133, 1065556, 9337841, 1085000]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[933637, 995965, 987557, 6534178, 15452443]"


In [21]:
result.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[1136958, 9859133, 1065556, 9337841, 1085000]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[933637, 995965, 987557, 6534178, 15452443]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [33]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [34]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()

0.0982793117246888

In [35]:
result.apply(lambda row: recall_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()

0.0014764167899478414

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code